# impots

In [3]:
from azure.storage.blob import BlobServiceClient

import pandas as pd
import openai
import json
import os
from tqdm import tqdm
from os import path 
from dotenv import load_dotenv

from langchain_community.chat_models import AzureChatOpenAI
from openai import AzureOpenAI
import warnings
warnings.filterwarnings("ignore")

# setting env variables

In [4]:
load_dotenv()

os.environ.update({
    "AZURE_OPENAI_API_KEY": os.getenv("openai_api_key"),
    "AZURE_OPENAI_ENDPOINT": os.getenv("azure_endpoint"),      
})

# Azure OpenAI, setting env variables 
gpt_model = os.getenv("openai_model_gpt_name")
embedding_model = os.getenv("openai_model_embd_name")
azure_openai_version = os.getenv("openai_api_version")
gpt_model = os.getenv("openai_model_gpt_name")
embedding_model_deployment = os.getenv("openai_model_embd_name")

# blob specifications
blob_key = os.getenv("blob_key")
connection_string = os.getenv("blob_connection_string")
container_name = os.getenv("blob_container_name")

# defining LLMs
llm = AzureChatOpenAI(openai_api_base=os.getenv("openai_api_base"), temperature=0.5,
                      deployment_name=gpt_model, openai_api_version=os.getenv("openai_api_version"))

# test LLM
x = llm.invoke("hello")
print(x)

content='Hello! How can I assist you today?'


# Implementing FAISS vector DB and similarity search
This code is designed to process a set of documents and a query, converting the text into numerical embeddings using OpenAI's GPT-3 model through Azure's API service. It then calculates the cosine similarity between the query's embedding and each document's embedding to identify documents that are similar to the query based on a predefined similarity threshold.

In [5]:
import openai
import numpy as np

# Set your OpenAI API key
openai.api_key = os.getenv("openai_api_key")

# Function to encode text using OpenAI GPT-3
client = AzureOpenAI(
      azure_endpoint = os.getenv("openai_api_base"), 
      api_key=os.getenv("openai_api_key"),  
      api_version=os.getenv("openai_api_version")
    )

def encode_text(text, engine=os.getenv("openai_model_embd_name")):
    response = client.embeddings.create(
        model=engine,
        input =text,
        # max_tokens=50
    )
    return np.array(response.data[0].embedding)

# Function to calculate cosine similarity between two vectors
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    return dot_product / (norm1 * norm2)

# Function to find similar documents
def find_similar_documents(query_embedding, document_embeddings, threshold):
    similar_documents = []
    for i, doc_embedding in enumerate(document_embeddings):
        similarity_score = cosine_similarity(query_embedding, doc_embedding)
        if similarity_score > threshold:
            similar_documents.append((i, similarity_score))
    return similar_documents

# Example text documents
documents = [
    "This is the first document",
    "This document is the second document",
    "And this is the third one",
    "Is this the first document",
]

# Example query
query = "this is the document"

# Encode text documents and query
document_embeddings = [encode_text(doc) for doc in documents]
query_embedding = encode_text(query)

# Find similar documents
threshold = 0.5  # Adjust threshold as needed
similar_documents = find_similar_documents(query_embedding, document_embeddings, threshold)

# Print similar documents
for doc_idx, similarity_score in similar_documents:
    print(f"Document Index: {doc_idx}, Similarity Score: {similarity_score}")


Document Index: 0, Similarity Score: 0.9343176002661533
Document Index: 1, Similarity Score: 0.9124196831436062
Document Index: 2, Similarity Score: 0.8174597176954618
Document Index: 3, Similarity Score: 0.8877400628933146
